In [1]:
%load_ext autoreload
%autoreload 2

import os; import sys; from pathlib import Path
root = Path(os.getcwd()).parents[0]
sys.path.append(str(root))


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import numpy as np
from src.data import post_process


In [3]:
# load raw data

df = pd.read_csv('../data/booli_10_pages_raw_20210506.csv')

display(
    df.head(3),
    f'shape = {df.shape}'
)


,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Bostadstyp,Driftskostnad,Våning,Byggår,Energiklass,BRF,adress,area_and_n_rooms,location,url,Avgift
0,5 200 000 kr,4 795 000 kr,+405 000 kr (+8.4 %),2021-05-05,148 600 kr/m²,12,Lägenhet,766 kr/mån,BV,1883.0,4,BRF Havssvalget 24,Skeppar­gatan 21,"35 m², 2 rum","Lägenhet, Östermalm",https://www.booli.se/bostad/732615,NaN
1,8 000 000 kr,6 695 000 kr,+1 305 000 kr (+19.5 %),2021-05-06,115 900 kr/m²,6,Lägenhet,599 kr/mån,2 tr,2019.0,NaN,BRF Tegeludden 16,Östhammars­gatan 70B,"69 m², 3 rum","Lägenhet, Gärdet",https://www.booli.se/bostad/4099014,3 876 kr/mån
2,5 010 000 kr,4 495 000 kr,+515 000 kr (+11.5 %),2021-05-05,106 600 kr/m²,12,Lägenhet,380 kr/mån,3 tr,1934.0,NaN,BRF Bojen 11,Kungs­holms Strand 141,"47 m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4200730,2 123 kr/mån


'shape = (283, 17)'

In [4]:
# post process data

df_pp = post_process(df)

display(
    df_pp.head(3),
    f'shape = {df_pp.shape}'
)


/Users/sebastian/miniconda3/envs/housing_price_prediction/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,price_ask,price_gain,sell_date,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
0,10750000.0,29.8,2021-04-28,3926.0,NaN,5.0,1934.0,99.0,3.0,108585.858586,0,1,0,0,0,0
1,4175000.0,18.6,2021-04-28,2716.0,220.0,5.0,1930.0,50.0,2.0,83500.000000,0,0,1,0,0,0
2,10950000.0,18.3,2021-04-28,4983.0,700.0,3.0,1932.0,112.0,4.5,97767.857143,0,0,0,0,0,1


'shape = (283, 16)'

In [5]:
# split covariates & target

X = df_pp.drop('sell_date', axis=1).dropna() 
y = X[['price_gain']]
X = X.drop(columns=['price_gain'])

#X.drop(index=y[y.price_gain==0].index, inplace=True)
#y.drop(index=y[y.price_gain==0].index, inplace=True)

display(
    X.head(3), f'shape = {X.shape}',
    y.head(3), f'shape = {y.shape}',
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
1,4175000.0,2716.0,220.0,5.0,1930.0,50.0,2.0,83500.000000,0,0,1,0,0,0
2,10950000.0,4983.0,700.0,3.0,1932.0,112.0,4.5,97767.857143,0,0,0,0,0,1
3,6250000.0,3551.0,850.0,5.0,2013.0,78.0,3.0,80128.205128,0,0,0,0,1,0


'shape = (151, 14)'

,price_gain
1,18.6
2,18.3
3,6.4


'shape = (151, 1)'

In [6]:
# split data
# sell order old --> recent, keep order by not shuffel and test on the most recent ones

train_ratio = 0.8; val_ratio = 0.1; test_ratio = 0.1
assert train_ratio + val_ratio + test_ratio == 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + val_ratio), shuffle=False) 

display(
    X_train.iloc[[0, -1]], 
    f'train shape X: {X_train.shape} y: {y_train.shape} ratio: {X_train.shape[0]/ X.shape[0]:.2f}',
    X_val.iloc[[0, -1]],
    f'val shape X: {X_val.shape} y: {y_val.shape} ratio: {X_val.shape[0]/ X.shape[0]:.2f}',
    X_test.iloc[[0, -1]],
    f'test shape X: {X_test.shape} y: {y_test.shape} ratio: {X_test.shape[0]/ X.shape[0]:.2f}'
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
1,4175000.0,2716.0,220.0,5.0,1930.0,50.0,2.0,83500.000000,0,0,1,0,0,0
226,15900000.0,3244.0,600.0,3.0,1939.0,124.0,5.0,128225.806452,1,0,0,0,0,0


'train shape X: (120, 14) y: (120, 1) ratio: 0.79'

,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
227,7200000.0,4405.0,517.0,1.0,1935.0,80.0,3.0,90000.000000,0,0,0,0,0,1
250,2950000.0,1979.0,575.0,3.0,1939.0,38.0,2.0,77631.578947,0,0,1,0,0,0


'val shape X: (15, 14) y: (15, 1) ratio: 0.10'

,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
252,6000000.0,2051.0,467.0,2.0,1882.0,55.0,2.0,109090.909091,0,0,0,0,0,1
281,6695000.0,3876.0,599.0,2.0,2019.0,69.0,3.0,97028.985507,0,1,0,0,0,0


'test shape X: (16, 14) y: (16, 1) ratio: 0.11'

In [7]:
# standardize numerical columns

num_cols, cat_cols = X.columns[:8], X.columns[8:]

scaler = preprocessing.StandardScaler()
scaler.fit(X_train[num_cols])
joblib.dump(scaler, '../data/scaler.pkl') # save scaler for future inference

X_train = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_train[num_cols]),
        columns=num_cols,
        index=X_train.index
    ),
    X_train[cat_cols]
    ],
    axis=1
)

X_val = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_val[num_cols]),
        columns=num_cols,
        index=X_val.index
    ),
    X_val[cat_cols]
    ],
    axis=1
)

X_test = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_test[num_cols]),
        columns=num_cols,
        index=X_test.index
    ),
    X_test[cat_cols]
    ],
    axis=1
)

display(
    X_train.iloc[[0, -1]],
    X_val.iloc[[0, -1]],
    X_test.iloc[[0, -1]],
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
1,-0.570703,0.013152,-1.361310,1.055211,-0.079023,-0.416907,-0.431661,-0.769884,0,0,1,0,0,0
226,3.339419,0.453371,0.387549,0.084902,0.178040,2.364560,2.615359,2.089924,1,0,0,0,0,0


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
227,0.438092,1.421353,0.005561,-0.885407,0.06379,0.710715,0.584012,-0.354268,0,0,0,0,0,1
250,-0.979223,-0.601320,0.272492,0.084902,0.17804,-0.867956,-0.431661,-1.145116,0,0,1,0,0,0


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_östermalm,location_gärdet,location_kungsholmen,location_vasastan,location_other,location_södermalm
252,0.037909,-0.54129,-0.224552,-0.400252,-1.450025,-0.228970,-0.431661,0.866422,0,0,0,0,0,1
281,0.269682,0.98030,0.382946,-0.400252,2.463043,0.297254,0.584012,0.095172,0,1,0,0,0,0


In [8]:
# save ml data

data = [X_train, X_val, X_test, y_train, y_val, y_test]
names = ['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test']

for d, n in zip(data, names):
    d.to_csv(f'../data/{n}.csv', index=False)
    